In [1]:
import os
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import BatchNormalization, Conv2D, Softmax, Reshape
from tensorflow.keras.models import Model
import numpy as np

In [2]:
from keras.models import load_model
model = load_model('/content/model.h5')

In [3]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 120, 120, 1)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 60, 60, 16)           144       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 60, 60, 16)           64        ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 60, 60, 16)           0         ['bn_Conv1[0][0]']      

In [4]:
# train_image = np.load("/content/ei-lab_human_detection-image-X_training.npy", mmap_mode='r')

In [5]:
# import json
# # Opening JSON file
# f = open('/content/ei-lab_human_detection-image-y_training.json')
# data = json.load(f)

# # Iterating through the json
# # list
# print(data)

# # Closing file
# f.close()

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
image_folder_path = '/content/data'

In [7]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 1  # You can adjust the batch size as needed
image_size = (160, 160)  # Specify the target size for resizing the images

generator = datagen.flow_from_directory(
    image_folder_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,  # Set to None if you don't have subdirectories representing class labels
    shuffle=False  # Set to False if you want to preserve the order of images
)

Found 0 images belonging to 0 classes.


In [8]:
#### rbg
# from tensorflow.keras.preprocessing import image
# def preprocess_image(img_path, target_size=(120,120)):
#     img = image.load_img(img_path, target_size=target_size)
#     img_array = image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0)
#     return img_array / 255.0

In [9]:
from tensorflow.keras.preprocessing import image
def preprocess_image(img_path, target_size=(120, 120), grayscale=True):
    img = image.load_img(img_path, target_size=target_size, color_mode='grayscale' if grayscale else 'rgb')
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array / 255.0

In [10]:
files = sorted(os.listdir(image_folder_path))

In [11]:
files

['image_test (01).jpg',
 'image_test (02).jpg',
 'image_test (03).jpg',
 'image_test (04).jpg',
 'image_test (05).jpg',
 'image_test (06).jpg',
 'image_test (07).jpg',
 'image_test (08).jpg',
 'image_test (09).jpg',
 'image_test (10).jpg',
 'image_test (11).jpg',
 'image_test (12).jpg',
 'image_test (13).jpg',
 'image_test (14).jpg',
 'image_test (15).jpg',
 'image_test (16).jpg',
 'image_test (17).jpg',
 'image_test (18).jpg',
 'image_test (19).jpg',
 'image_test (20).jpg',
 'image_test (21).jpg',
 'image_test (22).jpg',
 'image_test (23).jpg',
 'image_test (24).jpg',
 'image_test (25).jpg',
 'image_test (26).jpg',
 'image_test (27).jpg',
 'image_test (28).jpg',
 'image_test (29).jpg',
 'image_test (30).jpg',
 'image_test (31).jpg',
 'image_test (32).jpg',
 'image_test (33).jpg',
 'image_test (34).jpg',
 'image_test (35).jpg',
 'image_test (36).jpg',
 'image_test (37).jpg',
 'image_test (38).jpg',
 'image_test (39).jpg',
 'image_test (40).jpg',
 'image_test (41).jpg',
 'image_test (42

In [12]:
predictions = []
# Process images in sorted order
for filename in files:
    image_path = os.path.join(image_folder_path, filename)
    preprocessed_img = preprocess_image(image_path)
    prediction = model.predict(preprocessed_img)
    predictions.append((filename, prediction))


1/1 [==============================] - 0s 28ms/step


In [13]:
output=[]
names=[]
for i in predictions:
  x=list(i)
  output.append(x[1])
  names.append(x[0])

In [14]:
class Rectangle:
    def __init__(self, x, y, width, height, confidence):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.confidence = confidence

    def get_centroid(self):
        # Return the centroid as a list
        return [self.x + self.width / 2, self.y + self.height / 2]

def adjust_rectangle(rectangles, new_x, new_y, new_width, new_height, new_confidence, max_width=96, max_height=96):
    for rect in rectangles:
        is_overlapping = not (rect.x + rect.width < new_x or rect.y + rect.height < new_y or rect.x > new_x + new_width or rect.y > new_y + new_height)
        if is_overlapping:
            rect.x = min(rect.x, new_x)
            rect.y = min(rect.y, new_y)
            rect.width = min(max_width, max(rect.x + rect.width, new_x + new_width) - rect.x)
            rect.height = min(max_height, max(rect.y + rect.height, new_y + new_height) - rect.y)
            if new_confidence > rect.confidence:
                rect.confidence = new_confidence
            return False
    new_width = min(new_width, max_width - new_x)
    new_height = min(new_height, max_height - new_y)
    rectangles.append(Rectangle(new_x, new_y, new_width, new_height, new_confidence))
    return True

results = {}
# constant_width = 25   ##standing
constant_width = 60 # seated
constant_height = 105
threshold = 0.5
cell_width = int(120 / 12)
cell_height = int(120 / 12)

for k in range(len(output)):
    rectangles = []
    detections_found = False

    for i in range(12):
        for j in range(12):
            if output[k][0][i, j, 1] >= threshold:
                centroid_x = j * cell_width + cell_width / 2
                centroid_y = i * cell_height + cell_height / 2
                adjust_rectangle(rectangles, centroid_x - constant_width / 2, centroid_y - constant_height / 2, constant_width, constant_height, output[k][0][i, j, 1])
                detections_found = True

    # If no detections, set default values
    if not detections_found:
        results[names[k]] = [{'p_centriod': [0, 0], 'confidence_score': 0, 'p_human': 0}]
    else:
        # Assuming you want to store the details of the first rectangle for simplicity
        # Adjust as needed
        # for n in range(len(rectangles)):
        # first_detection = rectangles[0]
        # results[names[k]] = {
        #       'p_centriod': first_detection.get_centroid(),
        #       'confidence_score': first_detection.confidence,
        #       'p_human': 1
        #   }
        results[names[k]] = []

        for rectangle in rectangles:
            results[names[k]].append({
                'p_centriod': rectangle.get_centroid(),
                'confidence_score': rectangle.confidence,
                'p_human': 1
            })

# Display the results
for image_name, details in results.items():
    print(f"Image name: {image_name}, Details: {details}")


Image name: image_test (01).jpg, Details: [{'p_centriod': [0, 0], 'confidence_score': 0, 'p_human': 0}]
Image name: image_test (02).jpg, Details: [{'p_centriod': [0, 0], 'confidence_score': 0, 'p_human': 0}]
Image name: image_test (03).jpg, Details: [{'p_centriod': [40.0, 80.5], 'confidence_score': 0.9983426, 'p_human': 1}]
Image name: image_test (04).jpg, Details: [{'p_centriod': [70.0, 90.5], 'confidence_score': 0.9999214, 'p_human': 1}]
Image name: image_test (05).jpg, Details: [{'p_centriod': [0, 0], 'confidence_score': 0, 'p_human': 0}]
Image name: image_test (06).jpg, Details: [{'p_centriod': [0, 0], 'confidence_score': 0, 'p_human': 0}]
Image name: image_test (07).jpg, Details: [{'p_centriod': [25.0, 60.5], 'confidence_score': 0.99932784, 'p_human': 1}]
Image name: image_test (08).jpg, Details: [{'p_centriod': [55.0, 70.5], 'confidence_score': 0.9661458, 'p_human': 1}]
Image name: image_test (09).jpg, Details: [{'p_centriod': [95.0, 80.5], 'confidence_score': 0.9989834, 'p_human

In [15]:
# df1.head()

In [16]:
import pandas as pd
# df=pd.DataFrame.from_dict(results, orient='index')
# Flatten the dictionary to create a list of dictionaries
data = [{'Image name': key, **detection} for key, value in results.items() for detection in value]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

In [17]:
df.head()

,Image name,p_centriod,confidence_score,p_human
0,image_test (01).jpg,"[0, 0]",0.000000,0
1,image_test (02).jpg,"[0, 0]",0.000000,0
2,image_test (03).jpg,"[40.0, 80.5]",0.998343,1
3,image_test (04).jpg,"[70.0, 90.5]",0.999921,1
4,image_test (05).jpg,"[0, 0]",0.000000,0


In [18]:
### 96x96 rgb
# def draw_box(centroid):
#     # Calculate the top-left and bottom-right coordinates of the box
#     # top_left = (int(centroid[0] - box_size[0] / 2), int(centroid[1] - box_size[1] / 2))
#     # bottom_right = (int(centroid[0] + box_size[0] / 2), int(centroid[1] + box_size[1] / 2))
#     top_left = [int(centroid[0]) - 10, int(centroid[1])-40]
#     bottom_right = [int(centroid[0] + 15), int(centroid[1] + 35)]
#     if bottom_right[0] > 95:
#         bottom_right[0]=95
#     if bottom_right[1] > 95:
#         bottom_right[1]=95
#     if top_left[0] <0:
#         top_left[0] = 0
#     if top_left[1]<0 :
#        top_left[1] = 0
#     return top_left, bottom_right

In [19]:
def draw_box(centroid):
    # Calculate the top-left and bottom-right coordinates of the box
    # top_left = (int(centroid[0] - box_size[0] / 2), int(centroid[1] - box_size[1] / 2))
    # bottom_right = (int(centroid[0] + box_size[0] / 2), int(centroid[1] + box_size[1] / 2))
    top_left = [int(centroid[0]) - 20, int(centroid[1])-55]
    bottom_right = [int(centroid[0] + 20), int(centroid[1] + 55)]
    if bottom_right[0] > 120:
        bottom_right[0]=120
    if bottom_right[1] > 120:
        bottom_right[1]=120
    if top_left[0] <0:
        top_left[0] = 0
    if top_left[1]<0 :
       top_left[1] = 0
    return top_left, bottom_right

In [20]:
df.head()

,Image name,p_centriod,confidence_score,p_human
0,image_test (01).jpg,"[0, 0]",0.000000,0
1,image_test (02).jpg,"[0, 0]",0.000000,0
2,image_test (03).jpg,"[40.0, 80.5]",0.998343,1
3,image_test (04).jpg,"[70.0, 90.5]",0.999921,1
4,image_test (05).jpg,"[0, 0]",0.000000,0


In [21]:
top_left=[]
bottom_right=[]
for i in range(len(df)):
    if df['p_human'].iloc[i] == 0:
        tempx = [0, 0]
        tempy = [0, 0]
    else:
        # Assuming centroid is stored as a list [x, y] in the DataFrame
        centroid = df['p_centriod'].iloc[i]
        tempx, tempy = draw_box(centroid)
    top_left.append(tempx)
    bottom_right.append(tempy)

# If you want to add these lists as columns to the DataFrame:
df['p_left_top'] = top_left
df['p_right_bottom'] = bottom_right



In [22]:
df.head()

,Image name,p_centriod,confidence_score,p_human,p_left_top,p_right_bottom
0,image_test (01).jpg,"[0, 0]",0.000000,0,"[0, 0]","[0, 0]"
1,image_test (02).jpg,"[0, 0]",0.000000,0,"[0, 0]","[0, 0]"
2,image_test (03).jpg,"[40.0, 80.5]",0.998343,1,"[20, 25]","[60, 120]"
3,image_test (04).jpg,"[70.0, 90.5]",0.999921,1,"[50, 35]","[90, 120]"
4,image_test (05).jpg,"[0, 0]",0.000000,0,"[0, 0]","[0, 0]"


In [23]:
df.to_csv("normal_case_v98.csv")

jay

In [ ]:
# @title Default title text
threshold = 0.5
cell_width = 96 / 12  # 8 pixels
cell_height = 96 / 12  # 8 pixels
test={}
for k in range(len(output)):
  centroids = []
  confidences = []
  detected= 0
  for i in range(12):
      for j in range(12):
          if output[k][0][i, j, 1] >= threshold:
                # Calculate the centroid of the bounding box
                centroid_x = j * cell_width + cell_width / 2
                centroid_y = i * cell_height + cell_height / 2

                # Store the centroid
                centroids.append((centroid_x, centroid_y))
                confidences.append(output[k][0][i, j, 1])
                detected = 1
                print(f"Image name: {names[k]}, Centroid of all detections: ({int(centroid_x)}, {int(centroid_y)}), confidence {output[k][0][i, j, 1]}")
          else:
            print(f"Image name: {names[k]}, No detections")

In [ ]:
threshold = 0.5
cell_width = 96 / 12  # 8 pixels
cell_height = 96 / 12  # 8 pixels
test={}

for k in range(len(output)):
  temp={}
  centroids = []
  confidences = []
  detected= 0
  avg_x=0
  avg_y=0
  avg_confidence=0
  for i in range(12):
      for j in range(12):
          if output[k][0][i, j, 1] >= threshold:
                # Calculate the centroid of the bounding box
                centroid_x = j * cell_width + cell_width / 2
                centroid_y = i * cell_height + cell_height / 2

                # Store the centroid
                centroids.append((centroid_x, centroid_y))
                confidences.append(output[k][0][i, j, 1])
                detected = 1

    # Calculate the centroid of all detected points if any detections exist
  if centroids:
      avg_x = sum(x for x, y in centroids) / len(centroids)
      avg_y = sum(y for x, y in centroids) / len(centroids)
      avg_confidence = sum(confidences) / len(confidences) if confidences else 0  # Average confidence
      print(f"Image name: {names[k]}, Centroid of all detections: ({int(avg_x)}, {int(avg_y)}), confidence {avg_confidence}")
  else:
      print(f"Image name: {names[k]}, No detections")
  temp={"x_axis":int(avg_x),"y_axis":int(avg_y),"confidence":avg_confidence,"Detected":detected}
  test.update({names[k]:temp})
  detected=0

In [ ]:
  # temp={"name":names[k],"x_axis":int(avg_x),"y_axis":int(avg_y),"confidence":avg_confidence}
  # test.update(temp)
  print(test)

import pandas as pd
df=pd.DataFrame.from_dict(test, orient='index')

{'image_test (01).jpg': {'x_axis': 0, 'y_axis': 0, 'confidence': 0, 'Detected': 0}, 'image_test (02).jpg': {'x_axis': 92, 'y_axis': 56, 'confidence': 0.8777323365211487, 'Detected': 1}, 'image_test (03).jpg': {'x_axis': 24, 'y_axis': 64, 'confidence': 0.9934844225645065, 'Detected': 1}, 'image_test (04).jpg': {'x_axis': 44, 'y_axis': 66, 'confidence': 0.9242038428783417, 'Detected': 1}, 'image_test (05).jpg': {'x_axis': 0, 'y_axis': 0, 'confidence': 0, 'Detected': 0}, 'image_test (06).jpg': {'x_axis': 0, 'y_axis': 0, 'confidence': 0, 'Detected': 0}, 'image_test (07).jpg': {'x_axis': 16, 'y_axis': 52, 'confidence': 0.9539811164140701, 'Detected': 1}, 'image_test (08).jpg': {'x_axis': 44, 'y_axis': 56, 'confidence': 0.8729232847690582, 'Detected': 1}, 'image_test (09).jpg': {'x_axis': 62, 'y_axis': 62, 'confidence': 0.9548353354136149, 'Detected': 1}, 'image_test (10).jpg': {'x_axis': 56, 'y_axis': 61, 'confidence': 0.8903603076934814, 'Detected': 1}, 'image_test (11).jpg': {'x_axis': 16

In [ ]:
def draw_box(centroid):
    # Calculate the top-left and bottom-right coordinates of the box
    # top_left = (int(centroid[0] - box_size[0] / 2), int(centroid[1] - box_size[1] / 2))
    # bottom_right = (int(centroid[0] + box_size[0] / 2), int(centroid[1] + box_size[1] / 2))
    top_left = [int(centroid[0]) - 10, int(centroid[1])-40]
    bottom_right = [int(centroid[0] + 15), int(centroid[1] + 35)]
    if bottom_right[1] > 95:
        bottom_right[1]=95
    return top_left, bottom_right


In [ ]:
top_left, bottom_right =draw_box([21,56])

In [ ]:
# df.head()
top_left=[]
bottom_right=[]
for i in range(len(df)):
  x=df['x_axis'].iloc[i]
  y=df['y_axis'].iloc[i]
  temp_x,temp_y=draw_box([x,y])
  top_left.append(temp_x)
  bottom_right.append(temp_y)
# top_left=np.array(top_left)
# bottom_right=np.array(bottom_right)


In [ ]:
df['top_left']=top_left

df['bottom_right']=  bottom_right


In [ ]:
df = df.sort_index(ascending=True)


In [ ]:
df.to_csv("results.csv")